Tomar un ejemplo de los bots utilizados la clase 2 (uno de los dos bots implementados) y construir el propio.

Sacar conclusiones de los resultados.

In [1]:
import random 
import numpy as np
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

: 

: 

In [ ]:
!python -m spacy download es_core_news_sm

In [ ]:
import spacy

In [1]:
nlp = spacy.load("es_core_news_sm")

class Preprocessing:
    def __init__(self, dataset):
        self.dataset = dataset
        self.words = []
        self.classes = []
        self.doc_X = []
        self.doc_y = []

    def preprocess_clean_text(self, text):
        # Puedes agregar más lógica de limpieza de texto aquí si es necesario
        return text

    def tokenize_and_lemmatize(self):
        for intent in self.dataset["intents"]:
            for pattern in intent["patterns"]:
                tokens = nlp(self.preprocess_clean_text(pattern.lower()))
                for token in tokens:            
                    self.words.append(token.lemma_)
                self.doc_X.append(pattern)
                self.doc_y.append(intent["tag"])
            if intent["tag"] not in self.classes:
                self.classes.append(intent["tag"])

        self.words = sorted(set(self.words))
        self.classes = sorted(set(self.classes))

    def transform_data(self):
        training = []
        out_empty = [0] * len(self.classes)
        for idx, doc in enumerate(self.doc_X):
            text = []
            tokens = nlp(self.preprocess_clean_text(doc.lower()))
            for token in tokens:
                text.append(token.lemma_)
            
            bow = [1 if word in text else 0 for word in self.words]
            
            output_row = list(out_empty)
            output_row[self.classes.index(self.doc_y[idx])] = 1

            training.append([bow, output_row])

        random.shuffle(training)
        training = np.array(training, dtype=object)
        train_X = np.array(list(training[:, 0]))
        train_y = np.array(list(training[:, 1]))

        input_shape = (train_X.shape[1],)
        output_shape = train_y.shape[1]

        return train_X, train_y, input_shape, output_shape

class DNNModel:
    def __init__(self, input_shape, output_shape):
        self.model = Sequential([
            Dense(128, input_shape=input_shape, activation="relu"),
            Dropout(0.5),
            Dense(64, activation="relu"),
            Dropout(0.5),
            Dense(output_shape, activation="softmax")
        ])

        self.model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    def summary(self):
        print(self.model.summary())

    def fit(self, X, y, epochs=200, batch_size=5):
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size)

class ChatBot:
    def __init__(self, model, words, classes, dataset):
        self.model = model
        self.words = words
        self.classes = classes
        self.dataset = dataset

    def text_to_tokens(self, text):
        lemma_tokens = []
        tokens = nlp(text.lower())
        for token in tokens:
            lemma_tokens.append(token.lemma_)
        return lemma_tokens

    def bag_of_words(self, text):
        tokens = self.text_to_tokens(text)
        bow = [1 if word in tokens else 0 for word in self.words]
        return np.array(bow)

    def pred_class(self, text, thresh=0.2):
        bow = self.bag_of_words(text)
        words_recognized = sum(bow)
        
        return_list = []
        if words_recognized > 0:
            result = self.model.predict(np.array([bow]))[0]
            y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
            y_pred.sort(key=lambda x: x[1], reverse=True)

            for r in y_pred:
                return_list.append(self.classes[r[0]])
        return return_list

    def get_response(self, intents_list):
        tag = intents_list[0]
        list_of_intents = self.dataset["intents"]
        for i in list_of_intents:
            if i["tag"] == tag:
                result = random.choice(i["responses"])
                break
        return result

    def chat(self):
        while True:
            message = input("")
            intents = self.pred_class(message)
            if intents:
                response = self.get_response(intents)
                print(f"BOT: {response}")
            else:
                print("BOT: Perdón, no comprendo la pregunta.")


NameError: name 'spacy' is not defined